# Index Pro Features

* Demo Setup
* Filtered Indexes
* Compressed Indexes
* Included Columns
* Partitioned Indexes
* Columnstore Indexes

## Demo Setup

<div style="border:1px solid #888;padding:6px;background:#333333"> <strong>Note:</strong> This takes 30 minutes on my quad-core i7 laptop. Creating dbo.ReallyBigTable is not fast--so I create rowstore & columnstore copies up front to make it comparison easier later. </div>

* Create a DB to play in
* Create a few tables
* Fill them with some fun data

In [1]:
USE master
-- Create a DB to play in
DROP DATABASE IF EXISTS IndexProFeatures;
GO
CREATE DATABASE IndexProFeatures;
ALTER DATABASE IndexProFeatures 
    SET RECOVERY SIMPLE;

GO
USE IndexProFeatures
GO

--Create some tables

--Create some tables
CREATE TABLE dbo.WidgetQueue (
    QueueID bigint identity(1,1),
    QueueDate datetime2(0),
    QueueStatus char(1),
    SomeOtherStuff varchar(500),
    ExternalID uniqueidentifier,
    ExternalID2 uniqueidentifier CONSTRAINT DF_WidgetQueue_ExternalID2 DEFAULT NEWSEQUENTIALID(),
    NullCol01 int,
    NullCol02 int,
    NullCol03 int,
    NullCol04 int,
    NullCol05 int,
    NullCol06 int,
    NullCol07 nchar(4),
    NullCol08 nchar(4),
    NullCol09 uniqueidentifier,
    NullCol10 uniqueidentifier,
    CONSTRAINT PK_WidgetQueue PRIMARY KEY CLUSTERED (QueueID)
);

CREATE TABLE dbo.ReallyBigTable_rowstore (
    ReallyBigTableID bigint identity(1,1),
    StoreID tinyint,
    CustomerID int,
    ProductID int,
    TransactionTypeID int,
    CONSTRAINT PK_ReallyBigTable_rs PRIMARY KEY CLUSTERED (ReallyBigTableID)
);

CREATE TABLE dbo.ReallyBigTable_columnstore (
    ReallyBigTableID bigint,
    StoreID tinyint,
    CustomerID int,
    ProductID int,
    TransactionTypeID int);

--Use some data
INSERT INTO dbo.WidgetQueue (QueueDate, QueueStatus, SomeOtherStuff, ExternalID)
SELECT  QueueDate      = DATEADD(DAY,-(c1.object_id%10), CONVERT(date,GETDATE())),
        QueueStatus    = CASE c2. column_id%3
                           WHEN 0 THEN 'Q' 
                           WHEN 1 THEN 'S' 
                           WHEN 2 THEN 'L' 
                         END,
        SomeOtherStuff = c1.name + c2.name,
        ExternalID     = NEWID()
FROM sys.columns AS c1
CROSS JOIN sys.columns AS c2;
GO
INSERT INTO dbo.ReallyBigTable_rowstore (StoreID, CustomerID, ProductID, TransactionTypeID)
SELECT  StoreID             = c1.column_id,
        CustomerID          = c1.object_id,
        ProductID           = c2.object_id%10000,
        TransactionTypeID   = c2.system_type_id
FROM sys.columns AS c1
CROSS JOIN sys.columns AS c2;
GO 100

INSERT INTO dbo.ReallyBigTable_columnstore (ReallyBigTableID, StoreID, CustomerID, ProductID, TransactionTypeID)
SELECT ReallyBigTableID, StoreID, CustomerID, ProductID, TransactionTypeID
FROM ReallyBigTable_rowstore;

CREATE CLUSTERED COLUMNSTORE INDEX cci_ReallyBigTable ON dbo.ReallyBigTable_columnstore;


## Filtered Indexes


### Query without any indexes

* Look at number of reads, CPU, and execution time.

In [9]:
USE IndexProFeatures
GO
SET STATISTICS IO,TIME ON;

SELECT COUNT(*)
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

SET STATISTICS IO,TIME OFF;

### Query with a "regular" non-clustered index

* Look at number of reads, CPU, and execution time.

In [10]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_QueueStatus ON dbo.WidgetQueue;
CREATE INDEX ix_QueueStatus ON dbo.WidgetQueue (QueueStatus);

SET STATISTICS IO,TIME ON;

SELECT COUNT(*)
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

SELECT TOP 1 QueueID
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

SET STATISTICS IO,TIME OFF;

### Query with a filtered index

* Look at number of reads, CPU, and execution time.
* Open these queries in a new query window & look at the plans.

In [11]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_QueueStatus_filtered ON dbo.WidgetQueue;
CREATE INDEX ix_QueueStatus_filtered ON dbo.WidgetQueue (QueueStatus) WHERE QueueStatus = 'S';

SET STATISTICS IO,TIME ON;

SELECT COUNT(*)
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

SELECT TOP 1 QueueID
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

DECLARE @i char(1) = 'S';
SELECT TOP 1 QueueID
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = @i;

SET STATISTICS IO,TIME OFF;

### Maintenance costs

* Look at number of reads, CPU, and execution time.

In [12]:
USE IndexProFeatures
GO
SET STATISTICS IO,TIME ON;

ALTER INDEX ix_QueueStatus ON dbo.WidgetQueue REBUILD;
ALTER INDEX ix_QueueStatus_filtered ON dbo.WidgetQueue REBUILD;

SET STATISTICS IO,TIME OFF;

### Cleanup

In [13]:
USE IndexProFeatures
GO
SET STATISTICS IO,TIME OFF;
DROP INDEX IF EXISTS ix_QueueStatus ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_QueueStatus_filtered ON dbo.WidgetQueue;

## Compressed Indexes

### Query with a non-clustered index

* Look at number of reads, CPU, and execution time.

In [14]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_QueueStatus ON dbo.WidgetQueue;
CREATE INDEX ix_QueueStatus ON dbo.WidgetQueue (QueueStatus);

SET STATISTICS IO,TIME ON;

SELECT q.QueueStatus, COUNT(*)
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S'
GROUP BY q.QueueStatus;

SELECT TOP 10 *
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

SET STATISTICS IO,TIME OFF;

### How about with a row-compressed index?

* Look at number of reads, CPU, and execution time.

In [21]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_QueueStatus_row ON dbo.WidgetQueue;
CREATE INDEX ix_QueueStatus_row ON dbo.WidgetQueue (QueueStatus) WITH (DATA_COMPRESSION = ROW);
GO
SET STATISTICS IO,TIME ON;

SELECT q.QueueStatus, COUNT(*)
FROM dbo.WidgetQueue q WITH(index(ix_QueueStatus_row))
WHERE q.QueueStatus = 'S'
GROUP BY q.QueueStatus;

SELECT TOP 10 QueueID
FROM dbo.WidgetQueue q WITH(index(ix_QueueStatus_row))
WHERE q.QueueStatus = 'S';

SET STATISTICS IO,TIME OFF;

### With a page compressed index?

In [22]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_QueueStatus_page ON dbo.WidgetQueue;
CREATE INDEX ix_QueueStatus_page ON dbo.WidgetQueue (QueueStatus) WITH (DATA_COMPRESSION = PAGE);
GO

SET STATISTICS IO,TIME ON;

SELECT q.QueueStatus, COUNT(*)
FROM dbo.WidgetQueue q WITH(index(ix_QueueStatus_page))
WHERE q.QueueStatus = 'S'
GROUP BY q.QueueStatus;

SELECT TOP 10 QueueID
FROM dbo.WidgetQueue q WITH(index(ix_QueueStatus_page))
WHERE q.QueueStatus = 'S';

SET STATISTICS IO,TIME OFF;

### What about those GUID columns?
* ExternalID is a random GUID
* ExternalID2 is a sequential GUID
* Create some indexes at different compression levels... How well does "random" data compress?

In [23]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_ExternalID ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID_row ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID_page ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID2 ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID2_row ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID2_page ON dbo.WidgetQueue;
CREATE INDEX ix_ExternalID ON dbo.WidgetQueue (ExternalID);
CREATE INDEX ix_ExternalID_row ON dbo.WidgetQueue (ExternalID) WITH (DATA_COMPRESSION = ROW);
CREATE INDEX ix_ExternalID_page ON dbo.WidgetQueue (ExternalID) WITH (DATA_COMPRESSION = PAGE);
CREATE INDEX ix_ExternalID2 ON dbo.WidgetQueue (ExternalID2);
CREATE INDEX ix_ExternalID2_row ON dbo.WidgetQueue (ExternalID2) WITH (DATA_COMPRESSION = ROW);
CREATE INDEX ix_ExternalID2_page ON dbo.WidgetQueue (ExternalID2) WITH (DATA_COMPRESSION = PAGE);


SELECT  i.name, 
        ips.page_count, 
        ips.record_count, 
        ips.compressed_page_count
FROM sys.indexes i
CROSS APPLY sys.dm_db_index_physical_stats(db_id(), i.object_id, i.index_id, DEFAULT, 'DETAILED') ips
WHERE i.object_id = object_id('dbo.WidgetQueue')
AND i.name LIKE 'ix_ExternalID%'
AND index_level = 0;


### Maintenance
* Compression doesn't compress random data very well, so not much space savings
* How expensive are maintenance costs?

In [24]:
USE IndexProFeatures
GO
SET STATISTICS TIME ON;

PRINT '------->  QueueStatus';
ALTER INDEX ix_QueueStatus ON dbo.WidgetQueue REBUILD;
ALTER INDEX ix_QueueStatus_row ON dbo.WidgetQueue REBUILD;
ALTER INDEX ix_QueueStatus_page ON dbo.WidgetQueue REBUILD;

PRINT '------->  ExternalID';
ALTER INDEX ix_ExternalID ON dbo.WidgetQueue REBUILD;
ALTER INDEX ix_ExternalID_row ON dbo.WidgetQueue REBUILD;
ALTER INDEX ix_ExternalID_page ON dbo.WidgetQueue REBUILD;

PRINT '------->  ExternalID2';
ALTER INDEX ix_ExternalID2 ON dbo.WidgetQueue REBUILD;
ALTER INDEX ix_ExternalID2_row ON dbo.WidgetQueue REBUILD;
ALTER INDEX ix_ExternalID2_page ON dbo.WidgetQueue REBUILD;

SET STATISTICS TIME OFF;

### Cleanup

In [27]:
USE IndexProFeatures
GO
SET STATISTICS IO,TIME OFF;
DROP INDEX IF EXISTS ix_QueueStatus ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_QueueStatus_row ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_QueueStatus_page ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID_row ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID_page ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID2 ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID2_row ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_ExternalID2_page ON dbo.WidgetQueue;

## Included Columns

### With a non-covering non-clustered index

* Look at the execution plans for these two in a separate window

In [25]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_QueueStatus ON dbo.WidgetQueue;
CREATE INDEX ix_QueueStatus ON dbo.WidgetQueue (QueueStatus);

--Covered by ix_QueueStatus
SELECT QueueID, QueueStatus
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

--Not covered by ix_QueueStatus
SELECT QueueID, QueueStatus, ExternalID
FROM dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

### Add included columns to make it covering

* Look at the execution plans for these in anoter window

In [26]:
USE IndexProFeatures
GO
DROP INDEX IF EXISTS ix_QueueStatus_includes ON dbo.WidgetQueue;
CREATE INDEX ix_QueueStatusIncludes 
    ON dbo.WidgetQueue (QueueStatus)
    INCLUDE (ExternalID);

--Covered by ix_QueueStatus
SELECT QueueID, QueueStatus
FROM IndexProFeatures.dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

--Not covered by ix_QueueStatus
SELECT QueueID, QueueStatus, ExternalID
FROM IndexProFeatures.dbo.WidgetQueue q
WHERE q.QueueStatus = 'S';

### Cleanup

In [27]:
USE IndexProFeatures
GO
SET STATISTICS IO,TIME OFF;
DROP INDEX IF EXISTS ix_QueueStatus ON dbo.WidgetQueue;
DROP INDEX IF EXISTS ix_QueueStatus_includes ON dbo.WidgetQueue;

## Columnstore Indexes

### Let's query with rowstore first

* This isn't fast.... Does anyone want to tell a story while we wait?

In [28]:
USE IndexProFeatures
GO
SET STATISTICS IO,TIME ON;
SELECT StoreID, 
       COUNT(*)
FROM dbo.ReallyBigTable_rowstore
GROUP BY StoreID;

### But what if we use the columnstore copy of the table?


In [29]:
USE IndexProFeatures
GO
SET STATISTICS IO,TIME ON;

--We ran this earlier:
--   CREATE CLUSTERED COLUMNSTORE INDEX cci_ReallyBigTable ON dbo.ReallyBigTable_columnstore;

SELECT StoreID, 
       COUNT(*)
FROM dbo.ReallyBigTable_columnstore
GROUP BY StoreID;

SET STATISTICS IO,TIME OFF;

## Partitioned Tables Indexes

This is more than just creating an index.
* Partition Function
* Partition Scheme that uses the function
* Partitioned Table/Index that uses the scheme

In [0]:
USE IndexProFeatures
GO

DROP TABLE IF EXISTS dbo.PartitionedTable;
IF EXISTS (SELECT * FROM sys.partition_schemes WHERE name = N'ps_ByYear')
	DROP PARTITION SCHEME ps_ByYear;
IF EXISTS (SELECT * FROM sys.partition_functions WHERE name = N'pf_ByYear')
	DROP PARTITION FUNCTION pf_ByYear;

CREATE PARTITION FUNCTION pf_ByYear (DATE)
    AS RANGE RIGHT 
    FOR VALUES 
    ('2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01',
     '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01');

CREATE PARTITION SCHEME ps_ByYear
    AS PARTITION pf_ByYear 
    ALL TO ([Primary]);

CREATE TABLE dbo.PartitionedTable(
    TransactionID int identity(1,1),
    CustomerID int,
    TransactionDate DATE
) ON ps_ByYear(TransactionDate);


INSERT INTO dbo.PartitionedTable (CustomerID, TransactionDate)
VALUES  (1,'20161005'),
        (1,'20171005'),
        (1,'20181005'),
        (1,'20191005'),
        (1,'20201005'),
        (1,'20211005'),
        (2,'20161005'),
        (2,'20171005'),
        (2,'20181005'),
        (2,'20191005'),
        (2,'20201005'),
        (2,'20211005');
        